In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# df = pd.read_csv('archive/Resume/Resume.csv')
# df.head()

In [3]:
# from datasets import load_from_disk, Dataset
# dataset=Dataset.from_pandas(df)

In [4]:
# dataset=dataset.remove_columns('Unnamed: 0')

In [5]:
# dataset.save_to_disk('Chunked Dataset')

In [6]:
from datasets import load_from_disk, load_dataset
dataset=load_from_disk('Chunked Dataset')

In [7]:
df=pd.DataFrame(dataset)

In [8]:
df

,Short String,labels
0,hr administrator marketing associate hr admini...,HR
1,ication accomplished trainer cross server hosp...,HR
2,plans participation insurance pension plan per...,HR
3,ted official company page facebook facilitate ...,HR
4,egarding medical billing assistant general man...,HR
...,...,...
21635,order create excel spread sheets reports mana...,AVIATION
21636,common carriers prepare overseas containers s...,AVIATION
21637,osts goods acquired purchase order career deve...,AVIATION
21638,er naval aircraft squadron vs received general...,AVIATION


In [9]:
df.labels.value_counts()

INFORMATION-TECHNOLOGY    1204
HEALTHCARE                1117
CONSULTANT                1100
ADVOCATE                  1069
ENGINEERING               1062
PUBLIC-RELATIONS          1061
HR                        1044
FINANCE                   1036
ACCOUNTANT                1035
CONSTRUCTION              1026
BUSINESS-DEVELOPMENT      1012
AVIATION                  1001
BANKING                    986
CHEF                       960
FITNESS                    904
ARTS                       850
SALES                      841
DESIGNER                   833
DIGITAL-MEDIA              814
APPAREL                    801
TEACHER                    785
AGRICULTURE                565
AUTOMOBILE                 314
BPO                        220
Name: labels, dtype: int64

In [10]:
# from string import punctuation
# print(punctuation)

# import re

# def cleanResume(resumeText):
#     resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
#     resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) # remove non-ascii characters
#     resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
#     resumeText = re.sub(r'[0-9]+', '', resumeText)  #remove numbers
#     return resumeText.lower()

In [11]:
# df["Cleaned Resume"] = df["Resume_str"].apply(lambda x: cleanResume(x))
# len(df["Cleaned Resume"][1])

In [12]:
# import nltk
# nltk.download('stopwords')
# import string
# from nltk.corpus import stopwords
# from nltk import word_tokenize

# len(stopwords.words('english'))

In [13]:
# df=df.drop(['Resume_html','Resume_str'],axis=1)

In [14]:
# pd.set_option('max_colwidth', 800)
# df.head()

In [15]:
# stop_words = set(stopwords.words('english')) 

# def remove_stop_words(sentence): 
#   words = sentence.split() 
#   filtered_words = [word for word in words if word not in stop_words] 
#   return ' '.join(filtered_words)

In [16]:
# df["Cleaned Resume"] = df["Cleaned Resume"].apply(lambda x: remove_stop_words(x))

In [19]:
import math
has_nan = df['Short String'].isna().any().any()
has_nan

False

In [20]:
# df2=df.dropna(subset=['Short String'])

In [21]:
# df2.to_csv("Cleaned Dataset")

In [22]:
labels_to_id = {}
id_to_label = {}

for idx, label in enumerate(df.labels.unique()):
    labels_to_id[label] = idx
    id_to_label[idx] = label
    

In [23]:
# ds_df = df[df.Category == id_to_label[5]]
# nltk.download('punkt')
# resumes=""
# total_words = []
# for resume in ds_df["Cleaned Resume"]:
#     resumes += resume
#     words = word_tokenize(resume)
#     for word in words :
#         if word not in set(stopwords.words('english')) and word not in string.punctuation:
#             total_words.append(word)

# from wordcloud import WordCloud
# wordcloudimage = WordCloud( font_step=2 ,
#                             max_font_size=500,
#                             collocations = False,
#                             #collocation_threshold = 1 
#                           ).generate(resumes)

# plt.figure(figsize=(15,15))
# plt.imshow(wordcloudimage, interpolation='bilinear')
# plt.axis("off")
# plt.show()

In [24]:
id_to_label[5]

'BUSINESS-DEVELOPMENT'

In [32]:
from transformers import AutoTokenizer, DataCollatorWithPadding, BartTokenizerFast

checkpoint ="training-new"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [33]:
# def tokenize_function(example):
#     tokenized = tokenizer(example['Cleaned Resume'], truncation=True, padding=True, return_tensors='pt')
#     label = labels_to_id[example['Category']]
#     return {'tokenized':tokenized,'labels':label}

In [34]:
# tokenized={}
# def tokenize_function(example):
#     tokenized = tokenizer(example['Cleaned Resume'],return_tensors='pt',is_split_into_words= True)
# #     tokenized['attention_mask'] = tokenizer(example['Cleaned Resume'],return_tensors='pt',is_split_into_words= True)['attention_mask'].squeeze(0)
# #     tokenized['length'] = len(tokenized['input_ids'])
#     label = labels_to_id[example['Category']]
#     return {'input_ids':tokenized['input_ids'].squeeze(0),'attention_mask':tokenized['attention_mask'].squeeze(0),'labels':label}

In [35]:
dataset

Dataset({
    features: ['Short String', 'labels'],
    num_rows: 21640
})

In [36]:
def tokenize_function(example):
    return (tokenizer(example['Short String'],max_length=128,truncation=True,return_tensors='pt'))

In [37]:
# substring_length = 600

# def split_string(example):
#     labels=example['Category']
#     long_string=example['Cleaned Resume']
#     shorts = [long_string[i:i+substring_length] for i in range(0, len(long_string), substring_length)]
#     return {'Short String':shorts,'labels':labels}

In [38]:
# from datasets import Dataset
# dataset=Dataset.from_pandas(df)
# df2=dataset.map(split_string,remove_columns=dataset.column_names)
# df3 = pd.DataFrame(df2)
# df3 = df3.explode('Short String')
# dataset = Dataset.from_pandas(df3)
# dataset=dataset.remove_columns('__index_level_0__')
# dataset.save_to_disk('Chunked Dataset')

In [39]:
import torch
def label_encode(example):
    example['input_ids']=torch.tensor(example['input_ids']).squeeze(0)
    example['attention_mask']=torch.tensor(example['attention_mask']).squeeze(0)
    example['token_type_ids']=torch.tensor(example['token_type_ids']).squeeze(0)
    
    example['label'] = labels_to_id[example['labels']]
    return example

In [40]:
dataset=dataset.map(tokenize_function)#remove_columns=dataset.column_names)

Map:   0%|          | 0/21640 [00:00<?, ? examples/s]

In [41]:
dataset=dataset.map(label_encode)

Map:   0%|          | 0/21640 [00:00<?, ? examples/s]

In [42]:
remove_columns=['Short String', 'labels']
dataset=dataset.remove_columns(remove_columns)

In [43]:
dataset=dataset.train_test_split(test_size=0.1)

In [44]:
len(id_to_label)

24

In [45]:
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import AdamW
from accelerate import Accelerator

In [46]:
train_dataloader= DataLoader(
    dataset['train'],
    shuffle= True,
    collate_fn= data_collator,
    batch_size= 12,
)

eval_dataloader= DataLoader(
    dataset['test'],
    collate_fn= data_collator,
    batch_size= 12,
)

In [47]:
from transformers import AutoModelForSequenceClassification 
model= AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    id2label= id_to_label,
    label2id= labels_to_id,
)

In [48]:
optimizer= AdamW(model.parameters(), lr= 5e-5)

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [49]:
num_train_epochs= 30
num_update_steps_per_epoch= len(train_dataloader)
num_training_steps= num_train_epochs * num_update_steps_per_epoch

In [50]:
from transformers import Trainer, TrainingArguments, get_scheduler

lr_scheduler= get_scheduler(
    "cosine",
    optimizer= optimizer,
    num_warmup_steps= 0,
    num_training_steps= num_training_steps,
)

In [51]:
def loss_fn(outputs, labels):
    return torch.nn.CrossEntropyLoss()(outputs, labels)

In [52]:
import os
os.mkdir("./training-new1")
output_dir= "./training-new1/"
os.listdir(output_dir)

[]

In [ ]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    current_loss= []
    accuracy= []
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)
        
        

        predictions = outputs.logits.argmax(dim= -1)
        labels = batch["labels"]
        loss= loss_fn(outputs['logits'],labels)

        # Necessary to pad predictions and labels for being gathered
#         predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
#         labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

#         predictions_gathered = accelerator.gather(predictions)
#         labels_gathered = accelerator.gather(labels)
        current_loss.append(loss)
        acc= sum(predictions == labels) / len(labels)
        accuracy.append(acc)
        

        
    
    results = sum(accuracy)/ len(accuracy) #avg accuracy #metric.compute()
    loss= sum(current_loss)/ len(current_loss)
    print(
        f"epoch {epoch}:",
        {
            "accuracy": results,
            "loss" : loss
        }, "\n\n"
    )

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        



  0%|          | 0/48690 [00:00<?, ?it/s]

You're using a XLNetTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


epoch 0: {'accuracy': tensor(0.7159, device='cuda:0'), 'loss': tensor(1.0379, device='cuda:0')} 


epoch 1: {'accuracy': tensor(0.6901, device='cuda:0'), 'loss': tensor(1.1493, device='cuda:0')} 


epoch 2: {'accuracy': tensor(0.6952, device='cuda:0'), 'loss': tensor(1.2039, device='cuda:0')} 


epoch 3: {'accuracy': tensor(0.6791, device='cuda:0'), 'loss': tensor(1.2954, device='cuda:0')} 


epoch 4: {'accuracy': tensor(0.6952, device='cuda:0'), 'loss': tensor(1.2880, device='cuda:0')} 


epoch 5: {'accuracy': tensor(0.6763, device='cuda:0'), 'loss': tensor(1.4516, device='cuda:0')} 


epoch 6: {'accuracy': tensor(0.6961, device='cuda:0'), 'loss': tensor(1.4457, device='cuda:0')} 


epoch 7: {'accuracy': tensor(0.6823, device='cuda:0'), 'loss': tensor(1.5667, device='cuda:0')} 


epoch 8: {'accuracy': tensor(0.6897, device='cuda:0'), 'loss': tensor(1.4957, device='cuda:0')} 


epoch 9: {'accuracy': tensor(0.6994, device='cuda:0'), 'loss': tensor(1.5768, device='cuda:0')} 


epoch 10: 

In [ ]:
tokenizer.save_pretrained(output_dir)